In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/cellar/users/mikeyu/DeepTranslate/ddot')

import numpy as np
np.set_printoptions(linewidth=150)

In [3]:
# Import Ontology class from DDOT package
from ddot import Ontology

# Introduction


An ontology is a hierarchical arrangement of two types of nodes: (1)
genes at the leaves of the hierarchy and (2) terms at intermediate
levels of the hierarchy. The hierarchy can be thought of as directed
acyclic graph (DAG), in which each node can have multiple children or
multiple parent nodes. DAGs are a generalization of trees
(a.k.a. dendogram), where each node has at most one parent.

The DDOT Python library provides many functions for assembling,
analyzing, and visualizing ontologies.  The main functionalities are
implemented in an object-oriented manner by an "Ontology" class.  This
class can handle both ontologies that are data-driven as well as those
that are manually curated like the Gene Ontology.


In [4]:
# Import Ontology class from DDOT package
from ddot import Ontology

# Creating an Ontology object

An object of the Ontology class can be created in several ways. To demonstrate
this, we will build the following ontology

<img src="https://github.com/michaelkyu/ontology/blob/master/docs/toy_ontology.png?raw=true" width="500" align="left">

## Through the \_\_init\_\_ constructor

In [4]:
# Connections from child terms to parent terms
hierarchy = [('S3', 'S1'),
             ('S4', 'S1'),
             ('S5', 'S1'),
             ('S5', 'S2'),
             ('S6', 'S2'),
             ('S1', 'S0'),
             ('S2', 'S0')]

# Connections from genes to terms
mapping = [('A', 'S3'),
           ('B', 'S3'),
           ('C', 'S3'),
           ('C', 'S4'),
           ('D', 'S4'),
           ('E', 'S5'),
           ('F', 'S5'),
           ('G', 'S6'),
           ('H', 'S6')]

# Construct ontology
ont = Ontology(hierarchy, mapping)

## To and from a tab-delimited table or Pandas dataframe

In [5]:
ont.to_table('toy_ontology.txt')

,Parent,Child,EdgeType
0,S0,S1,Child-Parent
1,S0,S2,Child-Parent
2,S1,S3,Child-Parent
3,S1,S4,Child-Parent
4,S1,S5,Child-Parent
5,S2,S5,Child-Parent
6,S2,S6,Child-Parent
7,S3,A,Gene-Term
8,S3,B,Gene-Term
9,S3,C,Gene-Term


In [6]:
ont = Ontology.from_table('toy_ontology.txt')

## From the Network Data Exchange (NDEx). Requires creating a free user account at http://ndexbio.org/

In [7]:
# Replace with your own NDEx user account
ndex_server, ndex_user, ndex_pass = 'http://test.ndexbio.org', 'scratch', 'scratch'
# ndex_user, ndex_pass = 'ddot_test', 'ddot_test'

url, _ = ont.to_ndex(ndex_server=ndex_server, ndex_user=ndex_user, ndex_pass=ndex_pass)
print(url)

http://dev2.ndexbio.org/v2/network/cbe8c342-39f5-11e8-9da1-0660b7976219


In [8]:
ont2 = Ontology.from_ndex(url)
print(ont2)


8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: ['name', 'y_pos', 'Vis:Fill Color', 'Vis:Border Paint', 'x_pos', 'Label', 'Vis:Shape', 'NodeType', 'Size', 'Vis:Size', 'isRoot']
edge_attributes: ['EdgeType', 'Vis:Visible', 'Is_Tree_Edge', '2']


# Inspecting the structure of an ontology

An Ontology object contains seven attributes:

* ``genes`` : List of gene names
* ``terms`` : List of term names
* ``gene_2_term`` : dictionary mapping a gene name to a list of terms connected to that gene. Terms are represented as their 0-based index in ``terms``.
* ``term_2_gene`` : dictionary mapping a term name to a list or genes connected to that term. Genes are represented as their 0-based index in ``genes``.
* ``child_2_parent`` : dictionary mapping a child term to its parent terms.
* ``parent_2_child`` : dictionary mapping a parent term to its children terms.
* ``term_sizes`` : A list of each term's size, i.e. the number of unique genes contained within this term and its descendants. The order of this list is the same as ``terms``. For every ``i``, it holds that ``term_sizes[i] = len(self.term_2_gene[self.terms[i]])``

In [9]:
ont.genes

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [10]:
ont.terms

['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [11]:
ont.gene_2_term

{'A': [3],
 'B': [3],
 'C': [3, 4],
 'D': [4],
 'E': [5],
 'F': [5],
 'G': [6],
 'H': [6]}

In [12]:
ont.term_2_gene

{'S3': [0, 1, 2],
 'S4': [2, 3],
 'S5': [4, 5],
 'S6': [6, 7],
 'S0': [],
 'S1': [],
 'S2': []}

In [13]:
ont.child_2_parent

{'S1': ['S0'],
 'S2': ['S0'],
 'S3': ['S1'],
 'S4': ['S1'],
 'S5': ['S1', 'S2'],
 'S6': ['S2'],
 'S0': []}

In [14]:
ont.parent_2_child

{'S0': ('S1', 'S2'),
 'S1': ('S3', 'S4', 'S5'),
 'S2': ('S5', 'S6'),
 'S3': [],
 'S4': [],
 'S5': [],
 'S6': []}

Alternatively, the hierarchical connections can be viewed as a binary matrix, using `Ontology.connected()`

In [15]:
conn = ont.connected()
np.array(conn, dtype=np.int32)

array([[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]], dtype=int32)

A summary of an Ontology’s object, i.e. the number of genes, terms, and connections, can be printed `print(ont)`

In [16]:
print(ont)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: [2]


# Manipulating the structure of an ontology

DDOT provides several convenience functions for processing Ontologies into a desirable structure. Currently, there are no functions for adding genes and terms. If this is needed, then we recommend creating a new Ontology or manipulating the contents in a different library, such as NetworkX or igraph, and transforming the results into Ontology.

In [17]:
# Renaming genes and terms.
ont2 = ont.rename(genes={'A' : 'A_alias'}, terms={'S0':'S0_alias'})
ont2.to_table()

,Parent,Child,EdgeType
0,S0_alias,S1,Child-Parent
1,S0_alias,S2,Child-Parent
2,S1,S3,Child-Parent
3,S1,S4,Child-Parent
4,S1,S5,Child-Parent
5,S2,S5,Child-Parent
6,S2,S6,Child-Parent
7,S3,A_alias,Gene-Term
8,S3,B,Gene-Term
9,S3,C,Gene-Term


## Delete S1 and G while preserving transitive connections

<img src="https://github.com/michaelkyu/ontology/blob/master/docs/toy_ontology_delete_transitive.png?raw=true" width="250" align="left">

In [18]:
ont2 = ont.delete(to_delete=['S1', 'G'])
print(ont2)

7 genes, 6 terms, 8 gene-term relations, 6 term-term relations
node_attributes: []
edge_attributes: [2]


## Delete S1 and G (don't preserve transitive connections)

<img src="https://github.com/michaelkyu/ontology/blob/master/docs/toy_ontology_delete_not_transitive.png?raw=true" width="250" align="left">

In [19]:
ont2 = ont.delete(to_delete=['S1', 'G'], preserve_transitivity=False)
print(ont2)

7 genes, 6 terms, 8 gene-term relations, 3 term-term relations
node_attributes: []
edge_attributes: [2]


## Propagate gene-term connections

<img src="https://github.com/michaelkyu/ontology/blob/master/docs/toy_ontology_propagate_gene_term.png?raw=true" width="250" align="left">

In [20]:
ont2 = ont.propagate(direction='forward', gene_term=True, term_term=False)
print(ont2)

# Remove all transitive connections, and maintain only a parsimonious set of connections
ont3 = ont2.propagate(direction='reverse', gene_term=True, term_term=False)

8 genes, 7 terms, 27 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: [2]


## Propagate term-term connections

<img src="https://github.com/michaelkyu/ontology/blob/master/docs/toy_ontology_propagate_term_term.png?raw=true" width="250" align="left">

In [21]:
ont2 = ont.propagate(direction='forward', gene_term=False, term_term=True)
print(ont2)

# Remove all transitive connections, and maintain only a parsimonious set of connections
ont3 = ont2.propagate(direction='reverse', gene_term=False, term_term=True)

8 genes, 7 terms, 9 gene-term relations, 11 term-term relations
node_attributes: []
edge_attributes: [2]


## Take the subbranch consisting of all term and genes under S1

<img src="https://github.com/michaelkyu/ontology/blob/master/docs/toy_ontology_S1.png?raw=true" width="250" align="left">

In [22]:
ont2 = ont.focus(branches=['S1'])
print(ont2)

Genes and Terms to keep: 10
6 genes, 4 terms, 7 gene-term relations, 3 term-term relations
node_attributes: ['Original_Size']
edge_attributes: [2]


# Inferring a data-driven ontology

An ontology can also be inferred in a data-driven manner based on an input set of node-node similarities.

In [62]:
sim, genes = ont.flatten()
print(genes)
print(sim)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H']
[[ 1.4150375  1.4150375  1.4150375  0.4150375  0.4150375  0.4150375 -0.        -0.       ]
 [ 1.4150375  1.4150375  1.4150375  0.4150375  0.4150375  0.4150375 -0.        -0.       ]
 [ 1.4150375  1.4150375  2.         2.         0.4150375  0.4150375 -0.        -0.       ]
 [ 0.4150375  0.4150375  2.         2.         0.4150375  0.4150375 -0.        -0.       ]
 [ 0.4150375  0.4150375  0.4150375  0.4150375  2.         2.         1.         1.       ]
 [ 0.4150375  0.4150375  0.4150375  0.4150375  2.         2.         1.         1.       ]
 [-0.        -0.        -0.        -0.         1.         1.         2.         2.       ]
 [-0.        -0.        -0.        -0.         1.         1.         2.         2.       ]]


In [93]:
ont2 = Ontology.run_clixo(sim, 0.0, 1.0, square=True, square_names=genes)

In [97]:
print(ont2)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: ['CLIXO_score']


# Ontology alignment

<img src="https://github.com/michaelkyu/ontology/blob/master/docs/toy_ontology_alignment.png?raw=true" width="550" align="left">

In [95]:
## Make a second ontology

# Connections from child terms to parent terms
hierarchy = [('T3', 'T1'),
             ('T4', 'T1'),
             ('T1', 'T0'),
             ('T5', 'T0')]

# Connections from genes to terms
mapping = [('A', 'T3'),
           ('B', 'T3'),
           ('C', 'T3'),
           ('D', 'T4'),
           ('E', 'T4'),
           ('F', 'T4'),
           ('G', 'T5'),
           ('H', 'T5')]

# Construct ontology
ont_B = Ontology(hierarchy, mapping)

In [101]:
ont.align(ont_B)

collapse command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/collapseRedundantNodes /tmp/tmp69tzhltw
collapse command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/collapseRedundantNodes /tmp/tmpq7jbs_ag
Alignment command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/calculateFDRs /tmp/tmpdleaetmk /tmp/tmpwvbp55c8 0.05 criss_cross /tmp/tmp8bvabn36 100 40 gene


,Term,Similarity,FDR
Term,,,
S3,T3,0.985294,0.00
S1,T1,0.913608,0.00
S6,T5,0.910000,0.02
S0,T0,0.892982,0.01


# Construct ontotypes

In [102]:
# Genotypes can be represented as tuples of mutated genes
genotypes = [('A', 'B'),
             ('A', 'E'),
             ('A', 'H'),
             ('B', 'E'),
             ('B', 'H'),
             ('C', 'F'),
             ('D', 'E'),
             ('D', 'H'),
             ('E', 'H'),
             ('G', 'H')]

ontotypes = ont.get_ontotype(genotypes)
print(ontotypes)

   S0  S1  S2  S3  S4  S5  S6
0   0   0   0   2   0   0   0
1   0   0   0   1   0   1   0
2   0   0   0   1   0   0   1
3   0   0   0   1   0   1   0
4   0   0   0   1   0   0   1
5   0   0   0   1   1   1   0
6   0   0   0   0   1   1   0
7   0   0   0   0   1   0   1
8   0   0   0   0   0   1   1
9   0   0   0   0   0   0   2


In [103]:
# Genotypes can also be represented a genotype-by-gene matrix
import pandas as pd, numpy as np
genotypes_df = pd.DataFrame(np.zeros((len(genotypes), len(ont.genes)), np.float64),
                            index=['Genotype%s' % i for i in range(len(genotypes))],
                            columns=ont.genes)
for i, (g1, g2) in enumerate(genotypes):
    genotypes_df.loc['Genotype%s' % i, g1] = 1.0
    genotypes_df.loc['Genotype%s' % i, g2] = 1.0
print(genotypes_df)

ontotypes = ont.get_ontotype(genotypes_df, input_format='matrix')
print(ontotypes)

             A    B    C    D    E    F    G    H
Genotype0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
Genotype1  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
Genotype2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
Genotype3  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
Genotype4  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0
Genotype5  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
Genotype6  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0
Genotype7  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0
Genotype8  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype9  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0
            S0   S1   S2   S3   S4   S5   S6
Genotype0  0.0  0.0  0.0  2.0  0.0  0.0  0.0
Genotype1  0.0  0.0  0.0  1.0  0.0  1.0  0.0
Genotype2  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype3  0.0  0.0  0.0  1.0  0.0  1.0  0.0
Genotype4  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype5  0.0  0.0  0.0  1.0  1.0  1.0  0.0
Genotype6  0.0  0.0  0.0  0.0  1.0  1.0  0.0
Genotype7  0.0  0.0  0.0  0.0  1.0  0.0  1.0
Genotype8  0.0  0.0  0.0  0.0  0.0  1.0  1.0


# Conversions to NetworkX and igraph

In [104]:
G = ont.to_igraph()
print(G)

IGRAPH DN-- 15 16 --
+ attr: NodeType (v), name (v), EdgeType (e)
+ edges (vertex names):
A->S3, B->S3, C->S3, C->S4, D->S4, E->S5, F->S5, G->S6, H->S6, S1->S0, S2->S0,
S3->S1, S4->S1, S5->S1, S5->S2, S6->S2


In [105]:
G = ont.to_networkx()
print(G.nodes())
print(G.edges())

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
[('A', 'S3'), ('B', 'S3'), ('C', 'S3'), ('C', 'S4'), ('D', 'S4'), ('E', 'S5'), ('F', 'S5'), ('G', 'S6'), ('H', 'S6'), ('S1', 'S0'), ('S2', 'S0'), ('S3', 'S1'), ('S4', 'S1'), ('S5', 'S1'), ('S5', 'S2'), ('S6', 'S2')]


# Visualization in HiView (http://hiview.ucsd.edu)

In [106]:
url, _ = ont.to_ndex(ndex_server=ndex_server, ndex_user=ndex_user, ndex_pass=ndex_pass, layout='bubble-collect')

print('Enter this into the "NDEx Sever URL" field')
print(ndex_server)

print('Enter this into the "UUID of the main hierarchy" field at http://hiview.ucsd.edu:')
print(url.split('/')[-1])

Enter this into the "NDEx Sever URL" field
http://test.ndexbio.org
Enter this into the "UUID of the main hierarchy" field at http://hiview.ucsd.edu:
23f542c5-3a0a-11e8-9da1-0660b7976219


# After reading this tutorial

You should check out the list of functions of the [Ontology class](http://the-data-driven-ontology-toolkit-ddot.readthedocs.io/en/latest/ontology.html) and a list of [utility functions](http://the-data-driven-ontology-toolkit-ddot.readthedocs.io/en/latest/utils.html) that may help you build more concise pipelines